<a href="https://colab.research.google.com/github/sleepless-se/Colab/blob/master/page_satus_conv1d_rnn_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get -q -y install swig 
!apt-get install mecab
!apt-get install libmecab-dev
!apt-get install mecab-ipadic-utf8
!pip install mecab-python3
!pip install optuna

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 10 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (855 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3

In [4]:
import sys, pickle, os, json, keras, time, re
from sklearn.model_selection import KFold
from keras.datasets import boston_housing
from keras.models import Model,model_from_json
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.datasets import mnist
from keras import models
from keras import layers
import numpy as np
from google.colab import files
from keras.metrics import categorical_accuracy
from scipy.stats import zscore
import random
import pandas as pd
import matplotlib.pyplot as plt
import MeCab
import optuna
mecab = MeCab.Tagger("-Owakati")

Using TensorFlow backend.


In [0]:
def save_text_tokenizer(tokenizer,file_name):
  # saving
  with open(file_name+".pickle", 'wb') as handle:
      pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_text_tokenizer(file_name):
  # loading
  with open(file_name+".pickle", 'rb') as handle:
      return pickle.load(handle)

In [6]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/gdrive')

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
base_dir = "/content/gdrive/My Drive/data/status_filter/"
!ls "/content/gdrive/My Drive/data/status_filter/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
data		  keras_model.hdf5  page_st.csv
error.csv	  keras_model.json  text_token.pickle
error_domain.csv  logs		    weights.best.h5


In [7]:
page_file_name = 'page_st.csv'
target_columns = ['label','text']
dtype ={'label':str,'text':str} 
raw_df = pd.read_csv(base_dir + page_file_name,sep=',',error_bad_lines=False, dtype=dtype,engine='python')
raw_df = raw_df[~raw_df.duplicated()]
print('raw_df',raw_df.shape)
print(raw_df.columns)
print(raw_df['label'].value_counts())
raw_df.to_csv(base_dir+'page_st.csv',index=False)

raw_df (2452, 3)
Index(['label', 'url', 'text'], dtype='object')
no        1463
stock      692
list       139
top         96
block       25
login       25
except      12
Name: label, dtype: int64


In [0]:
def change_label(label):
  if label != 'stock' :label = 'no'
  return label

raw_df['label'] = raw_df['label'].apply(lambda x : change_label(x))

In [9]:
raw_df = raw_df.dropna()
raw_df = raw_df.sample(frac=1)
print(raw_df["label"].value_counts())
print('raw_df',raw_df.shape)
print(raw_df.columns)


no       1760
stock     692
Name: label, dtype: int64
raw_df (2452, 3)
Index(['label', 'url', 'text'], dtype='object')


In [10]:
text = 'sorry we can t this product page 　  　re sold out 404 whoops our bad it gone out of stock 切れ 在庫 なし 完売 ました 売り 切れ ページ 見つかり ません この 製品 もう 入手 できません'
def prepro(text:str)->list:
  line = ""
  text = mecab.parse(str(text)).replace("↵"," ")
  text = re.sub('{.*}','',text)
  words = text.lower()
  words = re.sub('\s{2,}'," ",words)
  return words
prepro(text)


'sorry we can t this product page re sold out 404 whoops our bad it gone out of stock 切れ 在庫 なし 完売 まし た 売り 切れ ページ 見つかり ませ ん この 製品 もう 入手 でき ませ ん '

In [0]:
raw_df['edit_text'] = raw_df['text'].apply(lambda x : prepro(x))
token = keras.preprocessing.text.Tokenizer(num_words=250)

In [12]:
raw_df['edit_text'].values[0:1]

array(['access denied you don \' t have permission to access " http :// www 1 . bloomingdales . com / shop / product / marc - by - marc - jacobs - domo - arigato - mini - packrat - color - block - backpack ?" on this server . reference # 18 . b 7 c 733 b 8 . 1551963353 . e 1 d 2 d 8 d '],
      dtype=object)

In [13]:
raw_df['edit_text'] = raw_df['text'].apply(lambda x : prepro(x))
regex = "^(sorry.? (we can.t|this (product|page)))|^(sorry we.re sold out)|^(404|whoops, our bad...|it.s gone|out of stock|sold out|在庫切れ|在庫なし|完売しました|売り切れ|ページが見つかりません.*|この製品はもう入手できない.|ci scusiamo per l'inconveniente.|sorry!|back to our favs)$|(((product|page|item|requested) (is |was )?)(not found|no longer|sold out|out of stock|not available))|(re (sorry|sold out)|(ご購入いただけません)|(404 error))|^(sold out\\s*){3,}|ARTICLE.*EN LIGNE|^so sorry.*|404 page not|Don.t Cry|.*is out of stock.|articolo non disponibile";
p = re.compile(regex)
raw_df['flag'] = raw_df['text'].apply(lambda x : 0 if p.match(x) == None else 1)
raw_df['edit_text'] = raw_df['edit_text'].fillna("")

token = keras.preprocessing.text.Tokenizer(num_words=250)
token.fit_on_texts(raw_df['edit_text'].values)
save_text_tokenizer(token,base_dir + 'text_token')
X = token.texts_to_matrix(raw_df['edit_text'].values)
flag = np.array(raw_df['flag'])
flag = flag.reshape(len(flag),1)
print('flag',flag.shape,'X',X.shape)
# X = np.concatenate((flag,X),axis=1)

# X = flag
# text_len = len(X[0])
# print(f'text_len:{text_len} samples:{len(X)}')
print('word_count',len(token.word_counts))
# print(token.word_counts)
# print(token.word_index)
X = np.array(X)
print('X',X.shape)

max_len = 1000
X_seq = token.texts_to_sequences(raw_df['text'].values)
X_seq = sequence.pad_sequences(X_seq,maxlen=max_len)
max_len = int(X_seq.shape[1])
print('max_len',max_len)
# X_seq = X_seq.reshape(X_seq.shape[0],1,X_seq.shape[1])
print('X_seq',X_seq.shape)

flag (2452, 1) X (2452, 250)
word_count 27729
X (2452, 250)
max_len 1000
X_seq (2452, 1000)


In [0]:
index2word =  {i+1:key for i, key in enumerate(token.word_index)}
# index2word

In [0]:
# label_token = keras.preprocessing.text.Tokenizer()
# label_token.fit_on_texts(raw_df['label'].values)
# y = label_token.texts_to_matrix(raw_df['label'].values)
# label_len = len(label_token.word_counts) + 1
raw_df['true'] = raw_df['label'].apply(lambda x : 1 if x == 'stock' else 0)
y = to_categorical(raw_df['true'].values)
# print('word_index',label_token.word_index)
# print('label_len',label_len)
# print('y shape',y.shape)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y, test_size=0.20, random_state=42)

In [0]:

def objective(trial):
  
#     k_layer = trial.suggest_int('k_layer', 0, 4)
#     l_layer = trial.suggest_int('l_layer', 0, 4)
#     n_layer = trial.suggest_int('n_layer', 0, 4)
#     m_layer = trial.suggest_int('m_layer', 0, 4)
    
    
#     embed_size = int(trial.suggest_discrete_uniform('embed_size', 800, 2000, 100))
#     embed_dim = int(trial.suggest_discrete_uniform('embed_dim', 32, 256, 1))
    
    conv1_units = int(trial.suggest_discrete_uniform('conv1_units', 12, 256, 1))
    conv2_units = int(trial.suggest_discrete_uniform('conv2_units', 12, 256, 1))
    conv1_pool = int(trial.suggest_discrete_uniform('conv1_pool', 2, 12, 1)) 
    conv2_pool = int(trial.suggest_discrete_uniform('conv2_pool', 2, 12, 1)) 
#     max_pool1_size = int(trial.suggest_discrete_uniform('max_pool1_size', 2, 256, 1)) 
#     max_pool2_size = int(trial.suggest_discrete_uniform('max_pool2_size', 2, 256, 1)) 

    lstm_units = int(trial.suggest_discrete_uniform('lstm_units', 2, 256, 1)) 
#     lstm2_ini = trial.suggest_categorical('lstm2_ini', ['zeros', 'ones', 'orthogonal']) 
    dropout_rate1 = trial.suggest_uniform('dropout_rate', 0, 1)
    dropout_rate2 = trial.suggest_uniform('dropout_rate', 0, 1)
    lstm_dropout = trial.suggest_uniform('dropout_rate', 0, 1)
    
    dense1_units = int(trial.suggest_discrete_uniform('dense1_units', 4, 256, 1))
    conv1_active = trial.suggest_categorical('conv1_active', ['relu', 'sigmoid', 'softmax']) 
    conv2_active = trial.suggest_categorical('conv2_active', ['relu', 'sigmoid', 'softmax']) 
#     last_dens_active = trial.suggest_categorical('last_dens_active', ['sigmoid', 'softmax']) 
#     dropout_rate2 = trial.suggest_uniform('dropout_rate', 0, 1)

    
    # create model
  
    embedding_layer = layers.Embedding(1500,output_dim=64)
    input_seq = layers.Input(shape=(max_len,))
    x = embedding_layer(input_seq)
    x = layers.Conv1D(conv1_units,9)(x)
    x = layers.Activation(conv1_active)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate1)(x)
    x = layers.MaxPooling1D(9)(x)
    x = layers.Conv1D(conv1_units,9)(x)
    x = layers.Activation(conv2_active)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate2)(x)
    x = layers.MaxPooling1D(9)(x)
    
    x = layers.Bidirectional(layers.CuDNNLSTM(lstm_units, return_sequences=True, kernel_initializer='Orthogonal'))(x) 
    x = layers.Dropout(lstm_dropout)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(2,activation='softmax')(x)
  

#     model = get_model(embed_size,embed_dim,l_layer,n_layer,m_layer, dens_active, last_dens_active,last_dens_units,conv_units,conv_pool,max_pool_size,lstm1_units,lstm2_units, dropout_rate1, dropout_rate2,lstm1_ini,letm2_ini)
    model = Model(input_seq,x)
    model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['categorical_accuracy'])
    
    # train model
    history = model.fit(X_train, y_train, 
                        verbose=0,
                        epochs=10,
                        validation_data=(X_test, y_test),
                        batch_size=128)

    # 学習モデルの保存
    model_json = model.to_json()
    with open(base_dir+'keras_model.json', 'w') as f_model:
        f_model.write(model_json)
    model.save_weights(base_dir+'keras_model.hdf5')

    # 最小値探索なので
    return -np.amax(history.history['val_categorical_accuracy'])


In [0]:
study = optuna.create_study(sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=500)
print('best_params')
print(study.best_params)
print('-1 x best_value')
print(-study.best_value)

print('\n --- sorted --- \n')
sorted_best_params = sorted(study.best_params.items(), key=lambda x : x[0])
for i, k in sorted_best_params:
    print(i + ' = ' + str(k))
# history
hist_df = study.trials_dataframe()
hist_df.to_csv(base_dir+"boston_svr.csv")


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


[I 2019-03-21 02:26:38,404] Finished trial#0 resulted in value: -0.837067208076446. Current best value is -0.837067208076446 with parameters: {'conv1_units': 253.0, 'conv2_units': 211.0, 'conv1_pool': 5.0, 'conv2_pool': 8.0, 'lstm_units': 75.0, 'dropout_rate': 0.744713382088526, 'dense1_units': 248.0, 'conv1_active': 'sigmoid', 'conv2_active': 'softmax'}.
[I 2019-03-21 02:26:55,035] Finished trial#1 resulted in value: -0.7800407314980345. Current best value is -0.837067208076446 with parameters: {'conv1_units': 253.0, 'conv2_units': 211.0, 'conv1_pool': 5.0, 'conv2_pool': 8.0, 'lstm_units': 75.0, 'dropout_rate': 0.744713382088526, 'dense1_units': 248.0, 'conv1_active': 'sigmoid', 'conv2_active': 'softmax'}.


In [0]:
0.8778004041757215

 --- sorted --- 

conv1_active = relu
conv1_pool = 12.0
conv1_units = 69.0
conv2_active = sigmoid
conv2_pool = 6.0
conv2_units = 113.0
dense1_units = 42.0
dropout_rate = 0.015408279269197522
embed_dim = 173.0
embed_size = 1300.0
lstm_units = 92.0

In [0]:
load_model = keras.models.load_model(base_dir +'keras_model.hdf5')
score = load_model.evaluate(X_test,y_test)
print('Test best loss:', score[0])
print('Test best accuracy:', score[1])

In [0]:
def get_model():
  activation = 'sigmoid'
  dropout_rate = 0.044713558274572573
  lstm_units = 178.0
  m_layer = 1
  mid_units = 96.0
  n_layer = 3
  optimizer = 'adam'
  embedding_size = 1500
  
  input_seq = layers.Input(shape = (784,))
  embedding_layer = layers.Embedding(embedding_size,output_dim=64)
  second_input_encoded = embedding_layer(input_seq) 
  for i in range(3):
    x = layers.Conv1D(mid_units, 9, activation=activation)(second_input_encoded)
    x = layers.MaxPooling1D(9)(x)
    x = layers.BatchNormalization()(x)
  x = layers.Bidirectional(layers.CuDNNLSTM(8, return_sequences=True, kernel_initializer='Orthogonal'))(x) 
  x = layers.Dropout(dropout_rate)(x)
  rnn_layer = layers.Bidirectional(layers.CuDNNLSTM(4, return_sequences=True, kernel_initializer='Orthogonal'))(x) 
  rnn_layer = layers.Flatten()(rnn_layer)
  output = layers.Dense(2,activation='softmax')(rnn_layer)
  model = Model(inputs=input_seq, outputs=output)
  model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['categorical_accuracy'])
  return model


In [0]:
EPOCHS = 150
BATCH_SIZE = 512

# stop when score is not improve
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=25)

# save best score
filepath = base_dir+"weights.best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')


# TensorBoard
# tsb = TensorBoard(log_dir=base_dir+'logs')

# reduce 
reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.01)
callbacks_list = [checkpoint,early_stopping,reduce]

# model = get_model(0.4)
# history = model.fit(X_train, y_train, validation_split=0.2,verbose=1,epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=callbacks_list)
# history_df = pd.DataFrame(history.history)
# history_df.plot()
# scores = []
# historys = []
# kf = KFold(n_splits=4, random_state=None, shuffle=False)
# for train_index, test_index in kf.split(X):
#   model = get_model()
#   train_data = [X[train_index], X_seq[train_index]]
#   test_data = [X[test_index], X_seq[test_index]]
#   history = model.fit(train_data, y[train_index], validation_split=0.2,verbose=0,epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=callbacks_list)
#   historys.append(history)
#   score = model.evaluate(test_data,y[test_index])
#   print(score)
#   scores.append(score)


In [0]:
model = get_model()
history = model.fit(X_train, y_train,validation_split=0.2,verbose=0,epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=callbacks_list)
load_model = keras.models.load_model(filepath)
score = load_model.evaluate(X_test, y_test)
print('score',score[1]*100,"%")
history_df = pd.DataFrame(history.history)
history_df[['val_categorical_accuracy','categorical_accuracy']].plot()

In [0]:
params = [512,1024,1500,2048]
for param in params:
  model = get_model(param)
  history = model.fit(X_train, y_train,validation_split=0.2,verbose=0,epochs=EPOCHS,batch_size=BATCH_SIZE,callbacks=callbacks_list)
  load_model = keras.models.load_model(filepath)
  score = load_model.evaluate(X_test, y_test)
  print('param',param)
  print('score',score[1]*100,"%")
  history_df = pd.DataFrame(history.history)
  history_df[['val_categorical_accuracy','categorical_accuracy']][35:].plot()

In [0]:

load_model = keras.models.load_model(filepath)
score = load_model.evaluate(X_test,y_test)
print('Test best loss:', score[0])
print('Test best accuracy:', score[1])

In [0]:
print("KFlod")
print("Sample",len(X))
print("Score",pd.DataFrame(scores).mean()[1]*100)
print("Std",pd.DataFrame(scores).std()[1]*100)


In [0]:
for i in range(len(historys)):
  df = pd.DataFrame(historys[i].history)
#   df.filter(regex=('loss*')).plot()
  df.filter(regex=('accuracy$')).plot()
  
plt.show()
  

In [0]:

import matplotlib.pyplot as plt

print(history.history.keys())
# summarize history for loss
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['val_loss', 'loss'], loc='upper left')
plt.show()

plt.plot(history.history['val_categorical_accuracy'])
plt.plot(history.history['categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['val_acc', 'acc'], loc='upper left')
plt.show()

In [0]:
len(test_index)

In [0]:
raw_df['pred'] = load_model.predict(X).argmax(axis=1)

In [0]:
error_df = raw_df.query("true != pred")
print(len(error_df))
error_df[['label','true','pred','url','text']].to_csv(base_dir+'error.csv',index=False)

In [0]:

def df_to_X(raw_df:pd.DataFrame):
  raw_df['edit_text'] = raw_df['text'].apply(lambda x : prepro(x))
  regex = "^(sorry.? (we can.t|this (product|page)))|^(sorry we.re sold out)|^(404|whoops, our bad...|it.s gone|out of stock|sold out|在庫切れ|在庫なし|完売しました|売り切れ|ページが見つかりません.*|この製品はもう入手できない.|ci scusiamo per l'inconveniente.|sorry!|back to our favs)$|(((product|page|item|requested) (is |was )?)(not found|no longer|sold out|out of stock|not available))|(re (sorry|sold out)|(ご購入いただけません)|(404 error))|^(sold out\\s*){3,}|ARTICLE.*EN LIGNE|^so sorry.*|404 page not|Don.t Cry|.*is out of stock.|articolo non disponibile";
  p = re.compile(regex)
  raw_df['flag'] = raw_df['text'].apply(lambda x : 0 if p.match(x) == None else 1)
  raw_df['edit_text'] = raw_df['edit_text'].fillna("")

  save_text_tokenizer(token,base_dir + 'text_token')
  X = token.texts_to_matrix(raw_df['edit_text'].values)
  flag = np.array(raw_df['flag'])
  flag = flag.reshape(len(flag),1)
#   print('flag',flag.shape,'X',X.shape)
  # X = np.concatenate((flag,X),axis=1)

  # X = flag
  # text_len = len(X[0])
  # print(f'text_len:{text_len} samples:{len(X)}')
#   print('word_count',len(token.word_counts))
#   print(token.word_counts)
#   print(token.word_index)
  X = np.array(X)
  return X

def get_split_num(text:str,num:int)->int:
    max_len = int(len(text))
    return int(max_len / num)
  
split = 10
df = pd.DataFrame()
pred_df = pd.DataFrame()
result = []
for i in range(split):
  df['text'] = raw_df['text'].apply(lambda x : x[get_split_num(x,split)*i:get_split_num(x,split)*(i+1)])
  X = df_to_X(df)
  first = load_model.predict(X).argmax(axis=1)
  result.append(first)
result

In [0]:
X = np.array(result)
X = X.T
print(X.shape)
X = X.reshape(len(X),split,1)
print(X.shape)
print(X[0:10:],y[0:10])

In [0]:
lstm_model = models.Sequential()
lstm_model.add(layers.LSTM(1,input_shape=(split,1)))
# lstm_model.add(layers.Flatten())
lstm_model.add(layers.Dense(2,activation='relu'))
lstm_model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['categorical_accuracy'])
history = lstm_model.fit(X,y,epochs=5,validation_split=0.2,batch_size=10)

In [0]:
pred = lstm_model.predict(X)

In [0]:
pred.argmax(axis=1)[0:10]

In [0]:
np.array(y).argmax(axis=1)[0:10]